In [1]:
from pyspark import SparkContext
from datetime import datetime
from dateutil.parser import parse
from pyspark.sql.functions import udf, to_date, to_utc_timestamp
import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1579212841840_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *


from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier


ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
station_path = 's3://msds694.proj/data/station.csv'
status_path = 's3://msds694.proj/data/status.csv'
trip_path = 's3://msds694.proj/data/trip.csv'
weather_path = 's3://msds694.proj/data/weather.csv'
population_path = 's3://msds694.proj/data/station_population.csv' 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
station_population=sc.textFile(population_path).map(lambda x:x.split(','))#.map(lambda x:[x[0],x[]])
station_population = station_population.toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
# station_population=ss.read.csv(population_path,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
# station_population = station_population.withColumnRenamed('_c7','population')
# station_population.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+---------+----------+----------+--------+-----------------+----------+----+
| id|                name|      lat|      long|dock_count|    city|installation_date|population| _c8|
+---+--------------------+---------+----------+----------+--------+-----------------+----------+----+
|  2|San Jose Diridon ...|37.329732|-121.90178|        27|San Jose|           8/6/13|      null|null|
|  3|San Jose Civic Ce...|37.330698|-121.88898|        15|San Jose|           8/5/13|      null|null|
|  4|Santa Clara at Al...|37.333988| -121.8949|        11|San Jose|           8/6/13|      null|null|
|  5|    Adobe on Almaden|37.331415| -121.8932|        19|San Jose|           8/5/13|      null|null|
|  6|    San Pedro Square|37.336721|-121.89407|        15|San Jose|           8/7/13|      null|null|
+---+--------------------+---------+----------+----------+--------+-----------------+----------+----+
only showing top 5 rows

In [115]:
station_population.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+
|station_id|population|
+----------+----------+
|        41|      9447|
|        42|      5535|
|        45|      9447|
|        46|     34557|
|        47|      4681|
|        48|      5535|
|        49|      1532|
|        50|      9447|
|        51|      9447|
|        39|      4681|
|        54|      1532|
|        55|     39050|
|        56|      9447|
|        57|     39050|
|        58|     10101|
|        59|     10101|
|        60|      5535|
|        61|     16156|
|        62|      1532|
|        63|     39050|
+----------+----------+
only showing top 20 rows

In [113]:
station_population = station_population[station_population['_6']=='San Francisco'].select('_1','_8')
station_population = station_population.withColumnRenamed('_1','station_id')
station_population.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|station_id|   _8|
+----------+-----+
|        41| 9447|
|        42| 5535|
|        45| 9447|
|        46|34557|
|        47| 4681|
+----------+-----+
only showing top 5 rows

In [114]:
station_population = station_population.withColumnRenamed('_8','population')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
#station_population_sf=station_population.dropna()
station_population=station_population.withColumn('population',station_population['population'].cast('integer'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
status=ss.read.csv(status_path,header=True)
status.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|              2|             25|2013/08/29 12:06:01|
|         2|              2|             25|2013/08/29 12:07:01|
|         2|              2|             25|2013/08/29 12:08:01|
|         2|              2|             25|2013/08/29 12:09:01|
|         2|              2|             25|2013/08/29 12:10:01|
+----------+---------------+---------------+-------------------+
only showing top 5 rows

In [9]:
status_rdd = sc.textFile(status_path)
weather_rdd = sc.textFile(weather_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
STATION_ID = 0
BIKES_AVAILABLE = 1
DOCKS_AVAILABLE = 2
TIME = 3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def change_col(x,col_num,func,**kwargs):
    x[col_num] = func(x[col_num],**kwargs)
    return x

# def makekey(x,*args):
#     key = tuple([item for idx,item in enumerate(x) if idx in args])
#     return (key,x)

# def flatten(x,iterations=1):
    
#     return sum(x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
status_rdd.cache().take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['station_id,bikes_available,docks_available,time', '2,2,25,2013/08/29 12:06:01']

In [13]:
status_rdd_processed = (status_rdd.filter(lambda x: not x.startswith('station_id'))
                                  .map(lambda x:x.split(','))
                                  .map(lambda x:x[:TIME]+[x[TIME].replace('/','-')])
                                  .map(lambda x: change_col(x,TIME,parse))
                                  .map(lambda x:x+[datetime(x[TIME].year,x[TIME].month,x[TIME].day,x[TIME].hour)]
                                       +[x[TIME].hour]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def toDoubleSafe(v):
    try:
        return float(v)
    except ValueError:
        return float(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
weather_rdd = sc.textFile(weather_path, 4)\
            .filter(lambda x: not x.startswith('date'))\
                                  .map(lambda x:x.split(','))\
                                  .map(lambda x:[x[0],toDoubleSafe(x[2]),toDoubleSafe(x[19])])\
                                  .map(lambda x: change_col(x,0,parse))#.map(lambda x:[x[0]]+[float(x[2])])
weather_rdd.take(3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[datetime.datetime(2013, 8, 29, 0, 0), 68.0, 0.0], [datetime.datetime(2013, 8, 30, 0, 0), 69.0, 0.0], [datetime.datetime(2013, 8, 31, 0, 0), 64.0, 0.0]]

In [16]:
weather_schema = StructType([
    StructField("date",DateType(),False),
    StructField("mean_temp",FloatType(),False),
    StructField("rain_identifier",FloatType(),False)])
    
weather_df = ss.createDataFrame(weather_rdd, schema=weather_schema)
weather_df.show(3)
# weather_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------------+
|      date|mean_temp|rain_identifier|
+----------+---------+---------------+
|2013-08-29|     68.0|            0.0|
|2013-08-30|     69.0|            0.0|
|2013-08-31|     64.0|            0.0|
+----------+---------+---------------+
only showing top 3 rows

In [17]:
weather_df_processed = weather_df.select(weather_df.date.cast(StringType()).alias('date'),
                                        weather_df.mean_temp.cast(DoubleType()).alias('temp_float'),
                                        weather_df.rain_identifier.cast(DoubleType()).alias('rain_level_float'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
weather_df_processed = weather_df_processed.groupBy('date').agg(mean('temp_float'),mean('rain_level_float'))
weather_df_processed = weather_df_processed.withColumnRenamed('avg(temp_float)','temp_float').withColumnRenamed('avg(rain_level_float)','rain_level_float')
weather_df_processed.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------------------+
|      date|temp_float|    rain_level_float|
+----------+----------+--------------------+
|2014-02-16|      56.0|0.021999999508261682|
|2014-02-22|      56.4|                 0.0|
|2014-05-27|      63.0|                 0.0|
|2014-12-13|      50.4|0.003999999910593033|
|2015-05-01|      54.2|                 0.0|
+----------+----------+--------------------+
only showing top 5 rows

In [19]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

weather_df_stats = weather_df_processed.select(
    _mean(col('rain_level_float')).alias('mean')
).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
mean_rain = weather_df_stats[0][0]
mean_rain

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.02012005447629855

In [21]:
from pyspark.sql.functions import *
check_rain = udf(lambda x: x > mean_rain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
weather_df_processed = weather_df_processed.select('date', 'temp_float', check_rain('rain_level_float') \
                                            .alias('rain_identifier'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
weather_df_processed.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, date: string, temp_float: string, rain_identifier: string]

In [24]:
status_rdd_processed.cache().take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['2', '2', '25', datetime.datetime(2013, 8, 29, 12, 6, 1), datetime.datetime(2013, 8, 29, 12, 0), 12], ['2', '2', '25', datetime.datetime(2013, 8, 29, 12, 7, 1), datetime.datetime(2013, 8, 29, 12, 0), 12], ['2', '2', '25', datetime.datetime(2013, 8, 29, 12, 8, 1), datetime.datetime(2013, 8, 29, 12, 0), 12]]

In [25]:
def is_weekend(date):
    weekday_name = date.strftime('%A')
    if weekday_name in ['Saturday','Sunday']:
        return True
    else:
        return False

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
status_rdd_week = status_rdd_processed.map(lambda x:x+[is_weekend(x[4])])\
                    .map(lambda x:[int(x[0]),int(x[1]),int(x[2])
                                   ,x[4],x[5],x[6]])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
status_rdd_week.cache().take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[2, 2, 25, datetime.datetime(2013, 8, 29, 12, 0), 12, False], [2, 2, 25, datetime.datetime(2013, 8, 29, 12, 0), 12, False], [2, 2, 25, datetime.datetime(2013, 8, 29, 12, 0), 12, False]]

In [28]:
schema = StructType([StructField("station_id", IntegerType(), True),
                     StructField("bikes_available", IntegerType(), False),
                     StructField("docks_available", IntegerType(), True),
                     StructField("time", DateType(), True),
                     StructField("hour", IntegerType(), False),
                     StructField("is_weekend", BooleanType(), True),
                     ])
df_status = ss.createDataFrame(status_rdd_week, schema)
df_status.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[station_id: int, bikes_available: int, docks_available: int, time: date, hour: int, is_weekend: boolean]

In [29]:
df_status = df_status.withColumnRenamed('time','date')
df_status.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+---------------+----------+----+----------+
|station_id|bikes_available|docks_available|      date|hour|is_weekend|
+----------+---------------+---------------+----------+----+----------+
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
+----------+---------------+---------------+----------+----+----------+
only showing top 5 rows

In [30]:
status_grop = df_status.groupBy('station_id','date','hour')\
    .agg(avg('docks_available'),avg('bikes_available'),min('is_weekend'))
status_grop.cache().show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+--------------------+--------------------+---------------+
|station_id|      date|hour|avg(docks_available)|avg(bikes_available)|min(is_weekend)|
+----------+----------+----+--------------------+--------------------+---------------+
|         2|2013-08-31|   4|                16.0|                11.0|           true|
|         2|2013-09-27|  19|                14.0|                13.0|          false|
|         2|2013-10-17|  16|  10.983333333333333|  16.016666666666666|          false|
|         2|2013-10-26|   3|                 6.0|                21.0|           true|
|         2|2013-11-03|  17|  10.566666666666666|  16.433333333333334|           true|
+----------+----------+----+--------------------+--------------------+---------------+
only showing top 5 rows

In [31]:
status_grop.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['station_id', 'date', 'hour', 'avg(docks_available)', 'avg(bikes_available)', 'min(is_weekend)']

In [32]:
status_grop = status_grop.withColumnRenamed('avg(docks_available)','docks_available')\
                .withColumnRenamed('avg(bikes_available)','bikes_available')\
                .withColumnRenamed('s_weekend','is_weekend')
status_grop.show(5)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+---------------+
|station_id|      date|hour|   docks_available|   bikes_available|min(is_weekend)|
+----------+----------+----+------------------+------------------+---------------+
|         2|2013-08-31|   4|              16.0|              11.0|           true|
|         2|2013-09-27|  19|              14.0|              13.0|          false|
|         2|2013-10-17|  16|10.983333333333333|16.016666666666666|          false|
|         2|2013-10-26|   3|               6.0|              21.0|           true|
|         2|2013-11-03|  17|10.566666666666666|16.433333333333334|           true|
+----------+----------+----+------------------+------------------+---------------+
only showing top 5 rows

In [33]:
status_grop.orderBy('station_id','date','hour').show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+---------------+
|station_id|      date|hour|   docks_available|   bikes_available|min(is_weekend)|
+----------+----------+----+------------------+------------------+---------------+
|         2|2013-08-29|  12|              25.0|               2.0|          false|
|         2|2013-08-29|  13| 24.30188679245283|  2.69811320754717|          false|
|         2|2013-08-29|  14|              25.0|               2.0|          false|
|         2|2013-08-29|  15|              25.0|               2.0|          false|
|         2|2013-08-29|  16|              25.0|               2.0|          false|
|         2|2013-08-29|  17| 24.09259259259259|2.9074074074074074|          false|
|         2|2013-08-29|  18|              24.0|               3.0|          false|
|         2|2013-08-29|  19|24.454545454545453|2.5454545454545454|          false|
|         2|2013-08-29|  20|25.403846153846153|1.5961538461538463|          false|
|   

In [59]:
status_grop=status_grop.withColumnRenamed("min(is_weekend)","is_weekend")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
status_grop.write.mode("overwrite")\
        .parquet("s3://msds694.proj/data/parquet/status_grouped")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
status_grop=ss.read.parquet("s3://msds694.proj/data/parquet/status_grouped")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Path does not exist: s3://msds694.proj/data/parquet/status_grouped;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 316, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: s3://msds694.proj/data/parquet/status_grouped;'



In [62]:
status_weather = status_grop.join(weather_df_processed,'date','left_outer')
status_weather.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+
|      date|station_id|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|
+----------+----------+----+------------------+------------------+----------+----------+---------------+
|2014-02-16|        62|   2|              11.0|               8.0|      true|      56.0|           true|
|2014-02-16|        10|   9|               7.0|               8.0|      true|      56.0|           true|
|2014-02-16|        14|   8|10.083333333333334| 8.916666666666666|      true|      56.0|           true|
|2014-02-16|        24|  18|               7.5|               7.5|      true|      56.0|           true|
|2014-02-16|        26|   8|               7.0|               8.0|      true|      56.0|           true|
|2014-02-16|        42|  23|               8.0|               7.0|      true|      56.0|           true|
|2014-02-16|        45|   9|               6.0|        

In [63]:
status_weather.orderBy('station_id','date','hour').show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+
|      date|station_id|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|
+----------+----------+----+------------------+------------------+----------+----------+---------------+
|2013-08-29|         2|  12|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  13| 24.30188679245283|  2.69811320754717|     false|      70.4|          false|
|2013-08-29|         2|  14|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  15|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  16|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  17| 24.09259259259259|2.9074074074074074|     false|      70.4|          false|
|2013-08-29|         2|  18|              24.0|        

In [64]:
def type_safe(v):
    try:
        return int(v)
    except ValueError:
        return str(v) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
trip_rdd = sc.textFile(trip_path).filter(lambda x: not x.startswith('id'))\
                                  .map(lambda x:x.split(','))\
                                  .map(lambda x: change_col(x,2,parse))\
                                  .map(lambda x: change_col(x,5,parse))\
                                  
trip_rdd.take(3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['4576', '63', datetime.datetime(2013, 8, 29, 14, 13), 'South Van Ness at Market', '66', datetime.datetime(2013, 8, 29, 14, 14), 'South Van Ness at Market', '66', '520', 'Subscriber', '94127'], ['4607', '70', datetime.datetime(2013, 8, 29, 14, 42), 'San Jose City Hall', '10', datetime.datetime(2013, 8, 29, 14, 43), 'San Jose City Hall', '10', '661', 'Subscriber', '95138'], ['4130', '71', datetime.datetime(2013, 8, 29, 10, 16), 'Mountain View City Hall', '27', datetime.datetime(2013, 8, 29, 10, 17), 'Mountain View City Hall', '27', '48', 'Subscriber', '97214']]

In [66]:
start = trip_rdd.map(lambda x: [type_safe(x[0]), x[2],x[2].hour, type_safe(x[4])])
end = trip_rdd.map(lambda x: [type_safe(x[0]), x[5], x[5].hour,type_safe(x[7])])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
startschema = StructType([
    StructField("trip_id",IntegerType(),False),
    StructField("start_date", DateType(),False),
    StructField("start_hour", IntegerType(),False),
    StructField("start_station_id",IntegerType(),False),
])

endschema = StructType([
    StructField("trip_id",IntegerType(),False),
    StructField("end_date", DateType(),False),
    StructField("end_hour", IntegerType(),False),
    StructField("end_station_id",IntegerType(),False),
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
start_df0 = ss.createDataFrame(start, startschema)
end_df0 = ss.createDataFrame(end, endschema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
start_result = start_df0.groupby('start_station_id', 'start_date','start_hour').count()
start_result.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+----------+-----+
|start_station_id|start_date|start_hour|count|
+----------------+----------+----------+-----+
|              49|2013-08-29|        19|    1|
|               3|2013-08-29|        20|    1|
|              45|2013-08-29|        17|    3|
|              77|2013-08-30|        11|    3|
|              66|2013-08-30|        10|    4|
|              47|2013-09-01|         1|    1|
|              28|2013-09-01|        10|    1|
|              47|2013-09-01|        16|    2|
|              48|2013-09-02|        11|    1|
|              76|2013-09-02|        15|    2|
|              28|2013-09-02|        20|    2|
|              49|2013-09-03|        20|    1|
|              47|2013-09-04|         9|    1|
|              65|2013-09-05|        16|    3|
|               8|2013-09-06|        20|    3|
|              73|2013-09-07|         7|    1|
|              71|2013-09-07|         9|    6|
|              50|2013-09-08|        10|    3|
|            

In [70]:
end_result = end_df0.groupby('end_station_id', 'end_date','end_hour').count()
end_result.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+--------+-----+
|end_station_id|  end_date|end_hour|count|
+--------------+----------+--------+-----+
|            65|2015-08-11|      18|    5|
|            63|2015-08-11|      16|    2|
|            59|2015-08-11|      15|    1|
|            54|2015-08-11|      13|    3|
|            67|2015-08-11|      12|    2|
|            54|2015-08-10|      18|    2|
|            84|2015-08-10|      17|    2|
|            50|2015-08-10|      15|    4|
|            28|2015-08-10|      15|    1|
|            12|2015-08-10|       9|    1|
|            82|2015-08-10|       8|    9|
|            45|2015-08-10|       8|    4|
|            70|2015-08-10|       5|    1|
|            69|2015-08-09|      17|    2|
|            68|2015-08-09|      13|    1|
|            63|2015-08-08|      13|    3|
|            12|2015-08-07|      21|    1|
|            42|2015-08-07|      19|    1|
|            48|2015-08-07|       8|    6|
|            33|2015-08-06|      22|    2|
+----------

In [71]:
end_result = end_result.withColumnRenamed('count','end_count')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
 result = start_result.join(end_result, (start_result['start_station_id'] == end_result['end_station_id']) 
                           & (start_result['start_hour'] == end_result['end_hour'])
                            & (start_result['start_date'] == end_result['end_date']),
                            'inner')
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
result.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+----------+-----+--------------+----------+--------+---------+
|start_station_id|start_date|start_hour|count|end_station_id|  end_date|end_hour|end_count|
+----------------+----------+----------+-----+--------------+----------+--------+---------+
|              49|2013-08-29|        19|    1|            49|2013-08-29|      19|        2|
|               3|2013-08-29|        20|    1|             3|2013-08-29|      20|        2|
|              66|2013-08-30|        10|    4|            66|2013-08-30|      10|        4|
+----------------+----------+----------+-----+--------------+----------+--------+---------+
only showing top 3 rows

In [74]:
result_df = result.select('start_station_id','start_date', 'start_hour', 'count', 'end_count')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:

result_df = result_df.withColumnRenamed('start_station_id','station_id').withColumnRenamed('start_date','date').withColumnRenamed('start_hour','hour').withColumnRenamed('count','start_count')
result_df.cache().show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+-----------+---------+
|station_id|      date|hour|start_count|end_count|
+----------+----------+----+-----------+---------+
|        65|2015-08-11|  18|          5|        5|
|        63|2015-08-11|  16|          4|        2|
|        54|2015-08-10|  18|          1|        2|
+----------+----------+----+-----------+---------+
only showing top 3 rows

In [76]:
status_weather.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+-----------------+----------+----------+---------------+
|      date|station_id|hour|   docks_available|  bikes_available|is_weekend|temp_float|rain_identifier|
+----------+----------+----+------------------+-----------------+----------+----------+---------------+
|2014-02-16|        62|   2|              11.0|              8.0|      true|      56.0|           true|
|2014-02-16|        10|   9|               7.0|              8.0|      true|      56.0|           true|
|2014-02-16|        14|   8|10.083333333333334|8.916666666666666|      true|      56.0|           true|
|2014-02-16|        24|  18|               7.5|              7.5|      true|      56.0|           true|
|2014-02-16|        26|   8|               7.0|              8.0|      true|      56.0|           true|
+----------+----------+----+------------------+-----------------+----------+----------+---------------+
only showing top 5 rows

In [77]:
final_df = status_weather.join(result_df,['station_id','date','hour'],'left_outer')
final_df.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|station_id|      date|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|start_count|end_count|
+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|        62|2014-02-16|   2|              11.0|               8.0|      true|      56.0|           true|       null|     null|
|        10|2014-02-16|   9|               7.0|               8.0|      true|      56.0|           true|       null|     null|
|        14|2014-02-16|   8|10.083333333333334| 8.916666666666666|      true|      56.0|           true|       null|     null|
|        24|2014-02-16|  18|               7.5|               7.5|      true|      56.0|           true|       null|     null|
|        26|2014-02-16|   8|               7.0|               8.0|      true|      56.0|           true|       

In [78]:
final_df.orderBy('start_count',ascending=False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|station_id|      date|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|start_count|end_count|
+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|        70|2015-01-28|   8| 9.516666666666667| 9.483333333333333|     false|      55.4|          false|         52|       15|
|        70|2015-01-07|   8|               9.7|               9.3|     false|      52.4|          false|         45|       21|
|        70|2015-02-18|   8|              12.6| 6.066666666666666|     false|      58.0|          false|         44|       16|
|        69|2015-06-23|   8| 9.666666666666666|13.333333333333334|     false|      64.8|          false|         44|        9|
|        69|2015-03-16|   8|17.016666666666666| 5.916666666666667|     false|      63.0|          false|       

In [79]:
final_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['station_id', 'date', 'hour', 'docks_available', 'bikes_available', 'is_weekend', 'temp_float', 'rain_identifier', 'start_count', 'end_count']

In [80]:
final_group = final_df.groupBy('station_id', 
                 'hour', 
                 'is_weekend', 
                 'temp_float',
                 'rain_identifier').agg(mean('docks_available')
                      ,mean('start_count')
                      ,mean('end_count')
                      ,mean('bikes_available'))
final_group.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+--------------------+----------------+--------------+--------------------+
|station_id|hour|is_weekend|temp_float|rain_identifier|avg(docks_available)|avg(start_count)|avg(end_count)|avg(bikes_available)|
+----------+----+----------+----------+---------------+--------------------+----------------+--------------+--------------------+
|        74|   8|      true|      56.0|           true|                10.0|            null|          null|                13.0|
|        36|   5|      true|      56.0|           true|                6.25|            null|          null|                8.75|
|         8|  23|      true|      56.0|           true|                 6.0|            null|          null|                 9.0|
|        37|   6|      true|      56.0|           true|                 5.0|            null|          null|                 6.0|
|         3|  18|      true|      56.0|           true|   5.083333333333333|            nu

In [81]:
final_group=final_group.withColumnRenamed('avg(docks_available)','docks_available').withColumnRenamed('avg(start_count)','start_count')\
.withColumnRenamed('avg(end_count)','end_count').withColumnRenamed('avg(bikes_available)','bikes_available')
final_group.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------------+-----------+---------+-----------------+
|station_id|hour|is_weekend|temp_float|rain_identifier|  docks_available|start_count|end_count|  bikes_available|
+----------+----+----------+----------+---------------+-----------------+-----------+---------+-----------------+
|        74|   8|      true|      56.0|           true|             10.0|       null|     null|             13.0|
|        36|   5|      true|      56.0|           true|             6.25|       null|     null|             8.75|
|         8|  23|      true|      56.0|           true|              6.0|       null|     null|              9.0|
|        37|   6|      true|      56.0|           true|              5.0|       null|     null|              6.0|
|         3|  18|      true|      56.0|           true|5.083333333333333|       null|     null|9.916666666666666|
+----------+----+----------+----------+---------------+-----------------+-----------+---

In [84]:
final_group = final_group.withColumn('total_docks',final_group['docks_available']+final_group['bikes_available'])
final_group.cache().show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------------+-----------+---------+-----------------+-----------+
|station_id|hour|is_weekend|temp_float|rain_identifier|  docks_available|start_count|end_count|  bikes_available|total_docks|
+----------+----+----------+----------+---------------+-----------------+-----------+---------+-----------------+-----------+
|        74|   8|      true|      56.0|           true|             10.0|       null|     null|             13.0|       23.0|
|        36|   5|      true|      56.0|           true|             6.25|       null|     null|             8.75|       15.0|
|         8|  23|      true|      56.0|           true|              6.0|       null|     null|              9.0|       15.0|
|        37|   6|      true|      56.0|           true|              5.0|       null|     null|              6.0|       11.0|
|         3|  18|      true|      56.0|           true|5.083333333333333|       null|     null|9.916666666666666|     

In [85]:
final_group = final_group.drop('docks_available')
final_group.cache().show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------+---------+-----------------+-----------+
|station_id|hour|is_weekend|temp_float|rain_identifier|start_count|end_count|  bikes_available|total_docks|
+----------+----+----------+----------+---------------+-----------+---------+-----------------+-----------+
|        74|   8|      true|      56.0|           true|       null|     null|             13.0|       23.0|
|        36|   5|      true|      56.0|           true|       null|     null|             8.75|       15.0|
|         8|  23|      true|      56.0|           true|       null|     null|              9.0|       15.0|
|        37|   6|      true|      56.0|           true|       null|     null|              6.0|       11.0|
|         3|  18|      true|      56.0|           true|       null|     null|9.916666666666666|       15.0|
+----------+----+----------+----------+---------------+-----------+---------+-----------------+-----------+
only showing top 5 rows

In [86]:
fc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#final_df.write.option("path", "/df").saveAsTable('final_data_frame')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'final_df' is not defined
Traceback (most recent call last):
NameError: name 'final_df' is not defined



In [ ]:
ss.sql("select * from parquet. `/df`").show()

In [3]:
df=ss.sql("select * from parquet. `s3://msds694.proj/data/parquet/final_group`")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_test=ss.read.parquet("s3://msds694.proj/data/parquet/final_group")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------+---------+------------------+-----------+
|station_id|hour|is_weekend|temp_float|rain_identifier|start_count|end_count|   bikes_available|total_docks|
+----------+----+----------+----------+---------------+-----------+---------+------------------+-----------+
|        34|  13|      true|      56.0|           true|       null|     null|               8.0|       23.0|
|        35|   4|      true|      56.0|           true|       null|     null|               6.0|       11.0|
|         9|  14|      true|      56.0|           true|       null|     null|               7.0|       15.0|
|        28|  10|      true|      56.0|           true|       null|     null|              12.0|       23.0|
|        37|  16|      true|      56.0|           true|       null|     null|3.8666666666666667|       11.0|
+----------+----+----------+----------+---------------+-----------+---------+------------------+-----------+
only showing top 5 

In [69]:
station_population_sf = station_population_sf.select('id','population')
station_population_sf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

35

In [73]:
station_population_sf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+
| id|population|
+---+----------+
| 41|      9447|
| 42|      5535|
| 45|      9447|
| 46|    34,557|
| 47|      4681|
| 48|      5535|
| 49|      1532|
| 50|      9447|
| 51|      9447|
| 39|      4681|
| 54|      1532|
| 55|    39,050|
| 56|      9447|
| 57|    39,050|
| 58|     10101|
| 59|    10,101|
| 60|      5535|
| 61|    16,156|
| 62|      1532|
| 63|    39,050|
+---+----------+
only showing top 20 rows

In [122]:
model_data=df_test.join(station_population,'station_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
model_data_final=model_data.na.fill(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
model_data_final.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------+---------+-----------------+------------------+----------+
|station_id|hour|is_weekend|temp_float|rain_identifier|start_count|end_count|  bikes_available|       total_docks|population|
+----------+----+----------+----------+---------------+-----------+---------+-----------------+------------------+----------+
|        41|  19|     false|      72.8|          false|        0.0|      0.0|             3.65|             14.55|      9447|
|        41|  13|     false|      65.0|          false|        2.0|      1.0|9.422222222222222|              15.0|      9447|
|        41|   8|     false|      68.8|          false|        1.0|      6.0|7.522222222222222|14.683333333333334|      9447|
|        41|  19|     false|      51.2|          false|        0.0|      0.0|              6.6|              15.0|      9447|
|        41|  11|     false|      55.8|           true|        0.0|      0.0|             12.0|              15.0|    

In [126]:
 #save to our s3 bucket
model_data_final.write.mode("overwrite")\
        .parquet("s3://msds694.proj/data/parquet/model_data_final")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# group-project assignment 2 stops here

# Machine learning

In [5]:
df=ss.read.parquet("s3://msds694.proj/data/parquet/model_data_final")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------+---------+-----------------+-----------+----------+
|station_id|hour|is_weekend|temp_float|rain_identifier|start_count|end_count|  bikes_available|total_docks|population|
+----------+----+----------+----------+---------------+-----------+---------+-----------------+-----------+----------+
|        49|  13|     false|      51.2|           true|        0.0|      0.0|              8.0|       19.0|      1532|
|        49|  23|     false|      78.2|          false|        0.0|      0.0|             10.0|       19.0|      1532|
|        49|   5|      true|      74.0|          false|        0.0|      0.0|4.283333333333333|       19.0|      1532|
|        49|  11|     false|      53.2|           true|        0.0|      0.0|9.216666666666667|       19.0|      1532|
|        49|  15|     false|      53.2|           true|        0.0|      0.0|              9.0|       19.0|      1532|
+----------+----+----------+----------+---------

In [7]:
dfrp=df.na.fill(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
dfrp1=dfrp.withColumn('population',dfrp['population'].cast('integer'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dfrp1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+------------------+------------------+------------------+------------------+----------+
|station_id|hour|is_weekend|temp_float|rain_identifier|       start_count|         end_count|   bikes_available|       total_docks|population|
+----------+----+----------+----------+---------------+------------------+------------------+------------------+------------------+----------+
|        49|  13|     false|      51.2|           true|               0.0|               0.0|               8.0|              19.0|      1532|
|        49|  23|     false|      78.2|          false|               0.0|               0.0|              10.0|              19.0|      1532|
|        49|   5|      true|      74.0|          false|               0.0|               0.0| 4.283333333333333|              19.0|      1532|
|        49|  11|     false|      53.2|           true|               0.0|               0.0| 9.216666666666667|              19.0|      1532|

In [10]:
#converting strings to numeric values
from pyspark.ml.feature import StringIndexer

def indexStringColumns(df, cols):
    #variable newdf will be updated several times
    newdf = df
    
    for c in cols:
        #For each given colum, fits StringIndexerModel.
        si = StringIndexer(inputCol=c, outputCol=c+"-num",stringOrderType="alphabetAsc")
        sm = si.fit(newdf)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-num" 
        #and then drops the original columns.
        #and drop the "-num" suffix. 
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

dfnumeric = indexStringColumns(dfrp1, ['rain_identifier'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_model=dfnumeric.select('station_id','hour','is_weekend','temp_float','total_docks','start_count','end_count'\
                   ,'rain_identifier','population','bikes_available')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Merging the data with Vector Assembler.
va = VectorAssembler(outputCol="features", inputCols=df_model.columns[0:-1]) #except the last col.
bikepoints = va.transform(df_model).select("features", "bikes_available")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
bikepoints.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+
|            features|  bikes_available|
+--------------------+-----------------+
|[49.0,13.0,0.0,51...|              8.0|
|[49.0,23.0,0.0,78...|             10.0|
|[49.0,5.0,1.0,74....|4.283333333333333|
+--------------------+-----------------+
only showing top 3 rows

In [13]:
bikepoints=bikepoints.withColumnRenamed('bikes_available','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Create Training and Test data.
biketsets = bikepoints.randomSplit([0.8, 0.2])
biketrain = biketsets[0].cache()
bikevalid = biketsets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Linear Regression

In [70]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
lrpredicts = lrModel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
lrevaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = lrevaluator.evaluate(lrpredicts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.429127000312138

In [74]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
cv = CrossValidator().setEstimator(lr).setEvaluator(lrevaluator).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(lr.maxIter, [1000]).addGrid(lr.regParam, [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
lrevaluator.evaluate(cvmodel.bestModel.transform(bikevalid))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.403741859193024

In [76]:
cvmodel.bestModel.transform(bikevalid).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+-----------------+
|            features|             label|       prediction|
+--------------------+------------------+-----------------+
|(9,[0,3,4,8],[39....|              15.0|9.084809534432035|
|(9,[0,3,4,8],[39....|16.008333333333333|9.045661339034192|
|(9,[0,3,4,8],[39....|              14.0|9.042649939388205|
|(9,[0,3,4,8],[39....|               9.0|8.997478944698388|
|(9,[0,3,4,8],[39....|              11.0|8.973387747530484|
|(9,[0,3,4,8],[39....| 8.204166666666667| 8.94026235142462|
|(9,[0,3,4,8],[39....|12.666666666666666|8.874011559212887|
|(9,[0,3,4,8],[39....|11.666666666666666|9.157117553677683|
|(9,[0,3,4,8],[39....|             11.59| 8.81378356629313|
|(9,[0,3,4,8],[39....|7.7555555555555555|8.771623971249301|
|(9,[0,3,4,8],[39....|10.841666666666667|8.677511277110566|
|(9,[0,3,4,8],[39....|11.941212121212121|8.729464376205472|
|(9,[0,3,4,8],[39....|              11.5|8.726452976559484|
|(9,[0,3,4,8],[39....|               8.0

In [43]:
v1=cvmodel.bestModel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
v1.to_csv("s3://msds694.proj/data/plot1.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Decision Tree

In [77]:
# Train the data.
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
dt = DecisionTreeRegressor(maxDepth=10, maxBins= 10, minInstancesPerNode=3, minInfoGain = 0)
dtmodel = dt.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
dtpredicts = dtmodel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
dtevaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = dtevaluator.evaluate(dtpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 2.93307

In [80]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
cv = CrossValidator().setEstimator(dt).setEvaluator(dtevaluator).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [9,10,11]).addGrid(dt.minInstancesPerNode, [3,4,5,6,7]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
dtevaluator.evaluate(cvmodel.bestModel.transform(bikevalid))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.9212622625208473

# Random Forest

In [22]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
rf = RandomForestRegressor(  ) 
rfmodel = rf.fit(biketrain)
rfpredicts = rfmodel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
rfevaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = rfevaluator.evaluate(rfpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 3.20065

In [25]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
cv = CrossValidator().setEstimator(rf).setEvaluator(rfevaluator).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [9,10,11]).addGrid(rf.minInstancesPerNode, [3,4,5,6,7]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
rfevaluator.evaluate(cvmodel.bestModel.transform(bikevalid))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.8672035891659977

In [28]:
cvmodel.bestModel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: double, prediction: double]

In [30]:
v1=cvmodel.bestModel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#v1=v1.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#v1.to_csv("s3://msds694.proj/data/plot2.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Gradient Boost Regressor

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'label', maxIter=10)
gbt_model = gbt.fit(biketrain)


In [ ]:
gbt_predictions = gbt_model.transform(bikevalid)
gbt_predictions.select('prediction', 'label', 'features').show(25)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
        labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = cv.fit(train_data)
predictions = cvModel.transform(test_data)
evaluator.evaluate(predictions)